In [ ]:
!curl -O https://raw.githubusercontent.com/deepjavalibrary/d2l-java/master/tools/fix-colab-gpu.sh && bash fix-colab-gpu.sh

## Prepare Java Kernel for Google Colab
Since Java is not natively supported by Colab, we need to run the following code to enable Java kernel on Colab.

1. Run the cell bellow (click it and press Shift+Enter),
2. (If training on CPU, skip this step) If you want to use the GPU with MXNet in DJL 0.10.0, we need CUDA 10.1 or CUDA 10.2.
Since Colab supports CUDA 10.1, we will have to follow some steps to setup the environment.
Refresh the page (press F5) and stay at Python runtime on GPU. Run the file fix-colab-gpu script.

And then ensure that you have switched to CUDA 10.1.
3. After that, switch runtime to Java and hardware to GPU.(Might require refreshing the page and switching runtime)

Now you can write Java code.

In [ ]:
!curl -O https://raw.githubusercontent.com/deepjavalibrary/d2l-java/master/tools/colab_build.sh && bash colab_build.sh

# GPU
:label:`sec_use_gpu`

在 :numref:`tab_intro_decade` 中，我们讨论了过去20年中计算能力的快速增长。简而言之，自2000年以来，GPU性能每十年增长1000倍。这提供了巨大的机会，但也表明需要提供这样的性能。

在本节中，我们开始讨论如何利用这种计算性能进行研究。首先是使用单个GPU，然后是如何使用多个GPU和多个服务器（具有多个GPU）。

具体来说，我们将讨论如何使用单个NVIDIA GPU进行计算。首先，确保至少安装了一个NVIDIA GPU。然后，下载[NVIDIA驱动和CUDA](https://developer.nvidia.com/cuda-downloads)并按照提示设置适当的路径。当这些准备工作完成，就可以使用`nvidia-smi`命令来查看显卡信息。


In [ ]:
%system nvidia-smi

你可能已经注意到 `DJL` 的 `NDArray` 看起来与 NumPy 的 `ndarray` 非常类似。但有一些关键区别，其中之一是 DJL 支持不同的硬件设备。

在 `DJL` 中，每个数组都有一个设备（`Device`）。当我们跨多个服务器部署作业时，有的机器有 GPU, 有的没有，这使事情会变得棘手。默认情况下，`DJL` 会检测软硬件环境，自动选择高性能的运算设备来提高计算效率。

要运行此部分中的程序，至少需要两个GPU。注意，对于大多数桌面计算机来说，这可能是奢侈的，但在云中很容易获得，例如，通过使用AWS EC2的多GPU实例。本节几乎所有的其他部分都不需要多个GPU。本节只是为了说明数据如何在不同的设备之间传递。

## 计算设备

我们可以指定用于存储和计算的设备，如 CPU 和 GPU.

CPU 和 GPU 可以用 `Device.cpu()` 和 `Device.gpu()` 表示。需要注意的是，`Device.gpu(1)` 只代表一个卡和相应的显存。如果有多个GPU，我们使用`gpu(i)`表示第$i$块GPU（$i$从0开始）。另外，`gpu(0)`和`gpu()`是等价的。


In [ ]:
%load ../utils/djl-imports

In [ ]:
System.out.println(Device.cpu());
System.out.println(Device.gpu());
System.out.println(Device.gpu(1));

我们可以查询可用gpu的数量。

In [ ]:
System.out.println("GPU count: " + Engine.getInstance().getGpuCount());
Device d = Device.gpu(1);

现在我们定义了一个方便的函数，这个函数允许我们在请求的GPU不存在的情况下运行代码。

In [ ]:
/* Return the i'th GPU if it exists, otherwise return the CPU */
public Device tryGpu(int i) {
    return Engine.getInstance().getGpuCount() > i ? Device.gpu(i) : Device.cpu();
}

/* Return all available GPUs or the [CPU] if no GPU exists */
public Device[] tryAllGpus() {
    int gpuCount = Engine.getInstance().getGpuCount();
    if (gpuCount > 0) {
        Device[] devices = new Device[gpuCount];
        for (int i = 0; i < gpuCount; i++) {
            devices[i] = Device.gpu(i);
        }
        return devices;
    }
    return new Device[]{Device.cpu()};
}

System.out.println(tryGpu(0));
System.out.println(tryGpu(3));

Arrays.toString(tryAllGpus())

## NDArray 与 GPU

当存在 GPU 的情况下，DJL 会优先使用 GPU。

In [ ]:
NDManager manager = NDManager.newBaseManager();
NDArray x = manager.create(new int[]{1, 2, 3});
x.getDevice();

需要注意的是，无论何时我们要对多个项进行操作，它们都必须在同一个设备上。例如，如果我们对两个张量求和，我们需要确保两个张量都位于同一个设备上，否则框架将不知道在哪里存储结果，甚至不知道在哪里执行计算。

### 存储在GPU上

有几种方法可以在GPU上存储张量。例如，我们可以在创建张量时指定存储设备。接下来，我们在第一个 `gpu` 上创建张量变量 `X`。在GPU上创建的张量只消耗这个GPU的显存。我们可以使用`nvidia-smi` 命令查看显存使用情况。一般来说，我们需要确保不创建超过GPU显存限制的数据。


In [ ]:
NDArray x = manager.ones(new Shape(2, 3), DataType.FLOAT32, tryGpu(0));
x

假设你至少有两个GPU，下面的代码将在第二个GPU上创建一个随机 `NDArray`。

In [ ]:
NDArray y = manager.randomUniform(-1, 1, new Shape(2, 3), DataType.FLOAT32, tryGpu(1));
y

### 复制

如果我们要计算 `X + Y`，我们需要决定在哪里执行这个操作。例如，如 :numref:`fig_copyto` 所示，我们可以将 `X` 传输到第二个GPU并在那里执行操作。
*不要*简单地`X`加上`Y`，
因为这会导致异常。运行时引擎不知道该怎么做：它在同一设备上找不到数据会导致失败。由于 `Y` 位于第二个 GPU 上，所以我们需要将 `X` 移到那里，然后才能执行相加运算。

![复制数据以在同一设备上执行操作。](http://d2l.ai/_images/copyto.svg)
:label:`fig_copyto`


In [ ]:
NDArray z = x.toDevice(tryGpu(1), true);
System.out.println(x);
System.out.println(z);

现在数据在同一个GPU上（`Z`和`Y`都在），我们可以将它们相加。


In [ ]:
y.add(z)

假设变量 `z` 已经存在于第二个 GPU 上。如果我们还是调用 `z.toDevice(Device.gpu(1))` 会发生什么？即使该变量已经存在于目标设备（第二个GPU）上，它仍将被复制并保存在新分配的显存中。有时，根据代码运行的环境不同，两个变量可能已经存在于同一设备上。因此，我们只想在变量存在于不同设备中时进行复制。在这种情况下，我们可以调用 `z.toDevice(Device.gpu(1), false)`。如果变量已经存在于指定的设备中，则这不会进行任何操作。

### 旁注

人们使用GPU来进行机器学习，因为他们希望运行速度快。但是在设备之间传输变量是缓慢的。所以我们希望你百分之百确定你想做一些缓慢的事情。如果深度学习框架只是自动复制而没有崩溃，那么你可能不会意识到你已经编写了一些缓慢的代码。

此外，在设备（CPU、GPU和其他机器）之间传输数据比计算慢得多。这也使得并行化变得更加困难，因为我们必须等待数据被发送（或者接收），然后才能继续进行更多的操作。这就是为什么拷贝操作要格外小心。根据经验，多个小操作比一个大操作糟糕得多。此外，除非你知道自己在做什么。否则，一次执行几个操作比代码中散布的许多单个操作要好得多。如果一个设备必须等待另一个设备才能执行其他操作，那么这样的操作可能会阻塞。这有点像排队订购咖啡，而不像通过电话预先订购时，当你在的时候发现咖啡已经准备好了。

最后，当我们打印张量或将张量转换为NumPy格式时。如果数据不在内存中，框架会首先将其复制到内存中，这会导致额外的传输开销。


## 神经网络与 GPU

类似地，神经网络模型可以指定设备。在接下来的几章中，我们将看到更多关于如何在GPU上运行模型的例子，因为它们将变得更加计算密集。

当输入为GPU上的 NDArray 时，模型将在同一GPU上计算结果。总之，只要所有的数据和参数都在同一个设备上，我们就可以有效地学习模型。在下面的章节中，我们将看到几个这样的例子。

## 小结

* 我们可以指定用于存储和计算的设备，例如CPU或GPU。
* 深度学习框架要求计算的所有输入数据都在同一设备上，无论是CPU还是GPU。
* 不经意地移动数据可能会显著降低性能。一个典型的错误如下：计算GPU上每个小批量的损失，并在命令行中将其报告给用户（或将其记录在NumPy `ndarray`中）时，将触发全局解释器锁，从而使所有GPU阻塞。最好是为GPU内部的日志分配内存，并且只移动较大的日志。

## 练习

1. 尝试一个更大的计算任务，比如大矩阵的乘法，看看CPU和GPU之间的速度差异。一个计算量很小的任务呢？
1. 我们应该如何在GPU上读写模型参数？
1. 测量计算1000个$100 \times 100$矩阵的矩阵乘法所需的时间，并记录输出矩阵的弗罗贝尼乌斯范数，一次记录一个结果，而不是在GPU上保存日志并仅传输最终结果。
1. 测量同时在两个GPU上执行两个矩阵乘法与在一个GPU上按顺序执行两个矩阵乘法所需的时间。提示：你应该看到近乎线性的缩放。